In [1]:
import os
import json
import torch
import httpx
from IPython.display import Markdown, display

In [2]:
HF_API_TOKEN = ""
MODEL_URL = "https://api-inference.huggingface.co/models/Qwen/Qwen2.5-0.5B-Instruct"

In [3]:
BASE_URL = "https://router.huggingface.co/v1/chat/completions"

HEADERS = {
    "Authorization": f"Bearer {HF_API_TOKEN}",
    "Content-Type": "application/json"
}

model_name = "Qwen/Qwen3-0.6B"


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [6]:
def run_qwen_chat(messages, max_new_tokens: int = 512) -> str:
    """
    Запускает Qwen3 в чат-режиме БЕЗ thinking.
    Возвращает только финальный текст ответа (строку).
    """
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True  # выключаем думающий режим ради скорости
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0
        )

    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):]
    content = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    return content

In [7]:
def extract_json(text: str):
    """
    Пытаемся вытащить первый корректный JSON из строки.
    На случай, если модель всё равно добавит лишний текст.
    """
    if not text:
        return None

    candidates = re.findall(r'\{[\s\S]*\}', text)
    for c in candidates:
        try:
            return json.loads(c)
        except Exception:
            continue
    return None

In [8]:
import re

def fix_category_by_rules(letter_text: str, llm_category: str) -> str:
    text = letter_text.lower()

    # 1. Жалоба/претензия
    if re.search(r"\bжалоб[аи]\b|\bпретензи[яи]\b|\bнедовольств[оа]\b|\bнарушени[ея]\b", text):
        return "complaint"

    # 2. Регулятор
    if re.search(r"банк россии|центральн[ыйого]+ банк|на основании .* федеральн[оого] закона|указани[ея] банк[а]? россии", text):
        return "regulator"

    # 3. Согласование
    if re.search(r"просим согласовать|на согласование|просим утвердить|утвержден[ие]|согласовани[ея]", text):
        return "approval"

    # 4. Партнёрское
    if re.search(r"предлагаем сотрудничество|партнёрск|партнерск|совместн(ый|ое|ые) проект|коммерческо[ея] предложени[ея]", text):
        return "partnership"

    # 5. Уведомление
    if re.search(r"настоящим уведомляем|сообщаем, что|доводим до вашего сведения", text):
        # если LLM не сказала complaint/regulator/approval, можно форсить notification
        if llm_category not in ("complaint", "regulator", "approval"):
            return "notification"

    # 6. По умолчанию вернём то, что сказала модель
    return llm_category


In [9]:
def preprocess_letter(text: str) -> str:
    """
    Убираем типичные приветствия/подписи, чтобы модель не залипала на них.
    Очень простая эвристика — для хакатона достаточно.
    """
    lines = [l.strip() for l in text.splitlines() if l.strip()]

    # уберём первые строки, если это приветствия
    GREETINGS = (
        "уважаемые",
        "здравствуйте",
        "добрый день",
        "добрый вечер",
        "доброе утро",
    )
    while lines and any(lines[0].lower().startswith(g) for g in GREETINGS):
        lines.pop(0)

    # уберём последние строки, если это подписи
    SIGNOFF = (
        "с уважением",
        "уважением",
        "искренне ваш",
        "искренне ваша",
        "best regards",
        "kind regards",
    )
    while lines and any(lines[-1].lower().startswith(s) for s in SIGNOFF):
        lines.pop()

    return "\n".join(lines)


In [10]:
def analyze_letter(thread_json_str: str) -> str:
    """
    Генерация ответа на последнее письмо в переписке.

    На вход подаётся ОДНА СТРОКА, в которой лежит история переписки
    в виде JSON (как правило, JSON-массив писем, отсортированных от
    первого к последнему). Функция возвращает текст ответа на
    последнее входящее письмо клиента.
    """

    system_prompt = '''Ты — виртуальный сотрудник банка, который ведёт переписку с клиентами.

Тебе на вход передают ОДНУ СТРОКУ, в которой содержится история переписки
в формате JSON. Как правило, это JSON-массив объектов-писем, отсортированных
по времени от первого к последнему.

Каждый объект-письмо может содержать поля:
- "subject" — тема письма;
- "body" — текст письма целиком;
- "category" — категория письма (info_request, complaint, regulator, partnership, approval, notification);
- "summary" — краткое описание содержания письма;
- "contract_number" — номер договора / счёта (может быть пустым);
- "laws" — список упомянутых нормативных актов;
- "emotion" — эмоциональная окраска письма (например: «нейтральное», «недовольство», «срочность / давление», «благодарность» и т.п.);
- "tone_of_voice" — стиль письма («формально», «полуформально», «неформально»).

Считай, что:
- письма в JSON расположены по времени ОТ ПЕРВОГО к ПОСЛЕДНЕМУ;
- ПОСЛЕДНИЙ объект в массиве — это последнее ВХОДЯЩЕЕ письмо клиента,
  на которое нужно подготовить ответ;
- предыдущие письма отражают историю диалога (как клиента, так и банка).

ТВОЯ ЗАДАЧА:
1. Внимательно проанализировать всю переписку, учитывая:
   - "body" и "summary" каждого письма;
   - "category" последнего письма — чтобы понять тип запроса
     (жалоба, запрос информации, уведомление, партнёрское предложение, запрос на согласование и т.д.);
   - "contract_number" — чтобы корректно ссылаться на договор / счёт, если это уместно;
   - "laws" — чтобы при необходимости корректно сослаться на упомянутые нормы;
   - "emotion" и "tone_of_voice" последнего письма — чтобы выбрать подходящий тон ответа.
2. Сформировать ОДИН связный ответ на ПОСЛЕДНЕЕ письмо от имени банка.

ТРЕБОВАНИЯ К ОТВЕТУ:
- Пиши на русском языке.
- Не упоминай JSON, поля JSON и технические детали.
- Не переписывай текст клиента дословно.
- Не пересказывай всю историю переписки — лишь при необходимости кратко
  отсылайся к прошлым договорённостям или сообщениям.
- Сохраняй профессиональный, вежливый и доброжелательный тон.
- Предпочитаемый стиль — деловой «формально» или «полуформально»,
  в зависимости от стиля клиента.
- Если "emotion" последнего письма указывает на жалобу, недовольство,
  раздражение или срочность / давление:
  - признай наличие проблемы;
  - при необходимости извинись от лица банка;
  - обозначь, какие действия уже предприняты или будут предприняты,
    и в какие ориентировочные сроки.
- Если "category" = "info_request":
  - по возможности дай конкретный ответ на вопрос;
  - если данных не хватает — чётко перечисли, что именно нужно уточнить
    (дата операции, номер договора, реквизиты и т.п.).
- Если "category" = "complaint":
  - признай, что получена жалоба / претензия;
  - обозначь шаги проверки и примерные сроки рассмотрения;
  - если по контексту уже есть результат — изложи его.
- Если "category" = "regulator":
  - используй максимально официальный тон;
  - при необходимости корректно ссылайся на упомянутые нормативные акты.
- Если "category" = "approval":
  - явно укажи, согласовано ли / не согласовано,
    на каких условиях или какие дополнительные шаги требуются.
- Если "category" = "partnership":
  - поблагодари за предложение и интерес к банку;
  - вежливо откажи или предложи следующий шаг (звонок, встречу,
    обмен материалами и т.п.).
- Если "category" = "notification":
  - при необходимости подтверди получение и фиксацию информации;
  - дополни ответ только теми действиями, которые логично вытекают
    из контекста переписки.

ФОРМАТ ВЫХОДА:
- Верни ТОЛЬКО текст письма-ответа, без каких-либо комментариев,
  без пояснений и без обрамляющего JSON.
- В конце письма можешь использовать стандартное вежливое завершение
  от имени банка.'''

    user_prompt = (
        "Ниже представлена история переписки в формате JSON (одна строка).\n"
        "Проанализируй её и сгенерируй ответ на последнее входящее письмо клиента от имени банка.\n\n"
        f"JSON:\n{thread_json_str}"
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    content = run_qwen_chat(messages, max_new_tokens=1024)

    print("=== RAW CONTENT ===")
    print(content)

    return content.strip()


In [11]:
sample_letter = """[
  {
    "subject": "Открытие расчётного счёта для ООО «Альфа»",
    "body": "Уважаемые коллеги! Просим предоставить перечень документов, необходимых для открытия расчётного счёта для ООО «Альфа», а также сообщить ориентировочные сроки рассмотрения заявки.",
    "category": "info_request",
    "summary": "Клиент запрашивает перечень документов и сроки рассмотрения заявки на открытие расчётного счёта для ООО.",
    "contract_number": "",
    "laws": [],
    "emotion": "нейтральное",
    "tone_of_voice": "формально"
  },
  {
    "subject": "Перечень документов для открытия счёта",
    "body": "Добрый день! Направляем перечень документов, необходимых для открытия расчётного счёта. После получения полного комплекта рассмотрение заявки обычно занимает до трёх рабочих дней.",
    "category": "notification",
    "summary": "Банк направил клиенту список необходимых документов и обозначил стандартный срок рассмотрения заявки.",
    "contract_number": "",
    "laws": [],
    "emotion": "нейтральное",
    "tone_of_voice": "формально"
  },
  {
    "subject": "Уточнение статуса и сроков по открытию счёта ООО «Альфа»",
    "body": "Коллеги, добрый день! Вчера мы направили полный пакет документов для открытия расчётного счёта ООО «Альфа». Подскажите, пожалуйста, текущий статус рассмотрения. Счёт нужен для проведения первых платежей по контракту уже на следующей неделе, поэтому просим по возможности ускорить процесс и сообщить ориентировочный срок открытия.",
    "category": "info_request",
    "summary": "Клиент уточняет статус рассмотрения документов на открытие счёта и просит по возможности ускорить процесс из-за скорых платежей по контракту.",
    "contract_number": "",
    "laws": [],
    "emotion": "срочность / давление",
    "tone_of_voice": "полуформально"
  }
]"""

result = analyze_letter(sample_letter)
print("\n=== MODEL REPLY ===")
print(result)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


=== RAW CONTENT ===
<think>
Хорошо, мне нужно сформировать ответ на последнее письмо клиента от имени банка. Последнее письмо в JSON-файле — это третий объект. Нужно проанализировать его.

Теперь посмотрю на содержание письма. Клиент уточняет статус рассмотрения документов на открытие счёта и просит ускорить процесс из-за скорых платежей. Статус "срочность / давление" указывает на необходимость ускорения.

Теперь нужно подобрать подходящий тон. Последнее письмо имеет эмоциональную окраску "срочность / давление", поэтому ответ должен быть вежливым и с уважением. Стиль "полуформально" — это подходящий выбор.

Ответ должен быть кратким и профессиональным. Нужно уточнить, что сроки рассмотрения ускорены, и что счёт будет готов на следующей неделе. Также можно добавить, что клиенту предложить уточнить сроки, если нужно.

Проверю, все ли поля в ответе соответствуют требованиям. Нет, в ответе нужно только текст письма, без комментариев. Убедиться, что нет лишних деталей и текст соответствует 